In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
a = 6378137.0
b = 6356752.314245

In [3]:
def calcN(a,b,phi):
    arriba = a**2
    abajo = (((a**2)*math.cos(phi)**2)+((b**2)*math.sin(phi)**2))**.5
    return arriba/abajo

In [4]:
def grados2rad(angulo):
    return angulo * math.pi/180

In [5]:
def conversiongeo2cart(phi,lam,h,a,b):
    phi = grados2rad(phi)
    lam = grados2rad(lam)
    N = calcN(a,b,phi)
    x =(N+h)* math.cos(phi)*math.cos(lam)
    y =(N+h)* math.cos(phi)*math.sin(lam)
    z =((N*(b**2/a**2))+h) * math.sin(phi)
    return x,y,z

In [6]:
def conversioncart2geo(x,y,z,a,b):
    Longitud =math.atan(y/x)
    Longitud = Longitud*180/math.pi-180
    e2= 1-(b**2/a**2)
    p= (x**2+y**2)**.5
    N_ =a
    h_ = (x**2+y**2+z**2)**.5-(a*b)**.5
    phi_ = math.atan((z/p)*(1-(e2*N_/(N_+h_)))**-1)
    condicion = True
    i = 0
    while condicion:
        Ni = a/(math.cos(phi_)**2+((b**2/a**2)*math.sin(phi_)**2))**.5
        hi = (p/math.cos(phi_))-Ni
        phii= math.atan((z/p)*(1-(e2*Ni/(Ni+hi)))**-1)
        condicion = (abs(hi-h_)>a*10e-20) and (abs(phii-phi_)>10e-20)
        h_ =hi
        phi_ = phii    
        i = i+1
    phi_ = phi_*180/math.pi
    return phi_,Longitud,h

In [7]:
datos = np.genfromtxt("datos_apoyo.txt",delimiter=",")

In [8]:
def gms2dec(grados,minutos,segundos):
    return grados+minutos/60+segundos/3600

In [11]:
def separar(numero):
    grados = numero[:2]
    minutos = numero[3:5]
    segundos = numero[5:7]+"."+numero[7:]
    return grados,minutos,segundos

In [12]:
def separar1(numero):
    grados = numero[:3]
    minutos = numero[4:6]
    segundos = numero[6:8]+"."+numero[8:]
    return grados,minutos,segundos

In [13]:
def dist2p(P1,P2):
    return ((P2[0]-P1[0])**2+(P2[1]-P1[1])**2+(P2[2]-P1[2])**2)**.5

In [14]:
data = []
for i in range(len(datos)):
    lat1g,lat1m,lat1s = separar(str(datos[i,0]))
    lat1 = gms2dec(float(lat1g),float(lat1m),float(lat1s))
    lon1g,lon1m,lon1s = separar1(str(datos[i,1]))
    lon1 = gms2dec(float(lon1g),float(lon1m),float(lon1s))
    h1 = datos[i,2]
    x1,y1,z1 = conversiongeo2cart(lat1,lon1,h1,a,b)
    lat2g,lat2m,lat2s = separar(str(datos[i,3]))
    lat2 = gms2dec(float(lat2g),float(lat2m),float(lat2s))
    lon2g,lon2m,lon2s = separar1(str(datos[i,4]))
    lon2 = gms2dec(float(lon2g),float(lon2m),float(lon2s))
    h2 = datos[i,5]
    x2,y2,z2 = conversiongeo2cart(lat2,lon2,h2,a,b)
    distancia_euc = dist2p(np.asarray([x1,y1,z1]),np.asarray([x2,y2,z2]))
    distanciaestacion = datos[i,6]
    punto1= datos[i,7]
    punto2=datos[i,8]
    renglon = [punto1,punto2,np.asarray([x1,y1,z1]),np.asarray([x2,y2,z2]),distancia_euc,distanciaestacion,distancia_euc-distanciaestacion]
    data.append(renglon)

In [15]:
npdata = np.asarray(data, dtype=object)

In [16]:
npdata

array([[10.0, 11.0,
        array([ -922886.72715334, -5951569.11657866,  2099281.85726845]),
        array([ -923608.04171829, -5951534.04024824,  2099036.70315641]),
        762.6438154315333, 762.566, 0.0778154315332813],
       [4.0, 3.0,
        array([ -924151.68281468, -5952372.84075829,  2096541.10163531]),
        array([ -924347.7783152 , -5952308.43195145,  2096629.12382028]),
        224.3877108229689, 224.384, 0.0037108229689124528],
       [11.0, 4.0,
        array([ -923608.04171829, -5951534.04024824,  2099036.70315641]),
        array([ -924151.68281468, -5952372.84075829,  2096541.10163531]),
        2688.337569843192, 2688.1995, 0.13806984319171534],
       [8.0, 4.0,
        array([ -921992.37525752, -5952223.15849636,  2097964.20772335]),
        array([ -924151.68281468, -5952372.84075829,  2096541.10163531]),
        2590.4140294106674, 2590.4349, -0.020870589332844247],
       [8.0, 9.0,
        array([ -921992.37525752, -5952223.15849636,  2097964.20772335]),
 

In [17]:
B = np.asarray([[(npdata[1,3][0]-npdata[1,2][0])/npdata[1,4],(npdata[1,3][1]-npdata[1,2][1])/npdata[1,4],(npdata[1,3][2]-npdata[1,2][2])/npdata[1,4],-(npdata[1,3][0]-npdata[1,2][0])/npdata[1,4],-(npdata[1,3][1]-npdata[1,2][1])/npdata[1,4],-(npdata[1,3][2]-npdata[1,2][2])/npdata[1,4],0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,0,-(npdata[4,3][0]-npdata[4,2][0])/npdata[4,4],-(npdata[4,3][1]-npdata[4,2][1])/npdata[4,4],-(npdata[4,3][2]-npdata[4,2][2])/npdata[4,4],(npdata[4,3][0]-npdata[4,2][0])/npdata[4,4],(npdata[4,3][1]-npdata[4,2][1])/npdata[4,4],(npdata[4,3][2]-npdata[4,2][2])/npdata[4,4],0,0,0,0,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-(npdata[0,3][0]-npdata[0,2][0])/npdata[0,4],-(npdata[0,3][1]-npdata[0,2][1])/npdata[0,4],-(npdata[0,3][2]-npdata[0,2][2])/npdata[0,4],(npdata[0,3][0]-npdata[0,2][0])/npdata[0,4],(npdata[0,3][1]-npdata[0,2][1])/npdata[0,4],(npdata[0,3][2]-npdata[0,2][2])/npdata[0,4],0,0,-1,0,0,0],[0,0,0,(npdata[3,3][0]-npdata[3,2][0])/npdata[3,4],(npdata[3,3][1]-npdata[3,2][1])/npdata[3,4],(npdata[3,3][2]-npdata[3,2][2])/npdata[3,4],-(npdata[3,3][0]-npdata[3,2][0])/npdata[3,4],-(npdata[3,3][1]-npdata[3,2][1])/npdata[3,4],-(npdata[3,3][2]-npdata[3,2][2])/npdata[3,4],0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0],[0,0,0,(npdata[2,3][0]-npdata[2,2][0])/npdata[2,4],(npdata[2,3][1]-npdata[2,2][1])/npdata[2,4],(npdata[2,3][2]-npdata[2,2][2])/npdata[2,4],0,0,0,0,0,0,0,0,0,-(npdata[2,3][0]-npdata[2,2][0])/npdata[2,4],-(npdata[2,3][1]-npdata[2,2][1])/npdata[2,4],-(npdata[2,3][2]-npdata[2,2][2])/npdata[2,4],0,0,0,0,-1,0],[0,0,0,0,0,0,-(npdata[5,3][0]-npdata[5,2][0])/npdata[5,4],-(npdata[5,3][1]-npdata[5,2][1])/npdata[5,4],-(npdata[5,3][2]-npdata[5,2][2])/npdata[5,4],0,0,0,0,0,0,(npdata[5,3][0]-npdata[5,2][0])/npdata[5,4],(npdata[5,3][1]-npdata[5,2][1])/npdata[5,4],(npdata[5,3][2]-npdata[5,2][2])/npdata[5,4],0,0,0,0,0,-1]])

In [18]:
B

array([[-0.87391373,  0.28704249,  0.39227721,  0.87391373, -0.28704249,
        -0.39227721,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , -1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.02821214, -0.35514399, -0.9343858 ,  0.02821214,
         0.35514399,  0.9343858 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -1.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.94580793, -0.04599307,  0.32145296,
        -0.94580793,  0.04599307, -0.32145296,  0.        ,  0.        ,
  

In [19]:
W = np.asarray([[npdata[1,6]],[npdata[4,6]],[npdata[0,6]],[npdata[3,6]],[npdata[2,6]],[npdata[5,6]]])

In [20]:
C = np.diag([.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.000004,.000004,.000004,.000004,.000004,.000004])

In [21]:
M = np.linalg.inv(np.dot(B,np.dot(C,B.T)))

In [22]:
K = np.dot(M,W)

In [23]:
V = np.dot(np.dot(C,B.T),K)

In [24]:
V

array([[ 2.11987279e-02],
       [-6.96285618e-03],
       [-9.51555920e-03],
       [ 5.33993082e-02],
       [ 6.13708362e-03],
       [ 4.26807156e-02],
       [ 1.85773756e-01],
       [-1.01331564e-01],
       [-1.83485649e-01],
       [-1.29942393e-03],
       [-1.63575923e-02],
       [-4.30369150e-02],
       [-5.20537364e-02],
       [ 2.53128683e-03],
       [-1.76915706e-02],
       [-2.07018631e-01],
       [ 1.15983642e-01],
       [ 2.11048978e-01],
       [ 1.94057855e-04],
       [ 3.68472339e-04],
       [ 4.40290126e-04],
       [ 7.54988388e-04],
       [-1.60991779e-04],
       [-2.68151992e-03]])

In [25]:
v=np.asarray([[npdata[1,3][0]],[npdata[1,3][1]],[npdata[1,3][2]],[npdata[1,2][0]],[npdata[1,2][1]],[npdata[1,2][2]],[npdata[4,2][0]],[npdata[4,2][1]],[npdata[4,2][2]],[npdata[4,3][0]],[npdata[4,3][1]],[npdata[4,3][2]],[npdata[0,2][0]],[npdata[0,2][1]],[npdata[0,2][2]],[npdata[0,3][0]],[npdata[0,3][1]],[npdata[0,3][2]],[npdata[1,5]],[npdata[4,5]],[npdata[0,5]],[npdata[3,5]],[npdata[2,5]],[npdata[5,5]]])

In [26]:
vprima = V-v

In [34]:
vprima

array([[ 9.24347800e+05],
       [ 5.95230842e+06],
       [-2.09662913e+06],
       [ 9.24151736e+05],
       [ 5.95237285e+06],
       [-2.09654106e+06],
       [ 9.21992561e+05],
       [ 5.95222306e+06],
       [-2.09796439e+06],
       [ 9.21980797e+05],
       [ 5.95207741e+06],
       [-2.09834767e+06],
       [ 9.22886675e+05],
       [ 5.95156912e+06],
       [-2.09928187e+06],
       [ 9.23607835e+05],
       [ 5.95153416e+06],
       [-2.09903649e+06],
       [-2.24383806e+02],
       [-4.10187632e+02],
       [-7.62565560e+02],
       [-2.59043415e+03],
       [-2.68819966e+03],
       [-2.05744838e+03]])

In [27]:
comp = dist2p(np.asarray([vprima[0,0],vprima[1,0],vprima[2,0]]),np.asarray([vprima[3,0],vprima[4,0],vprima[5,0]]))

In [35]:
[vprima[0,0],vprima[1,0],vprima[2,0]]

[924347.7995139271, 5952308.424988592, -2096629.1333358348]

In [36]:
vprima[18]

array([-224.38380594])

In [28]:
comp+vprima[18]

array([8.72987681e-06])

In [29]:
latitud_r=gms2dec(51,15,11582)
longitud_r=gms2dec(-114,6,1.76988)
h_r = 1127.345
xr,yr,zr=conversiongeo2cart(latitud_r,longitud_r,h_r,a,b)
receptor = np.asarray([[xr],[yr],[zr]])
satellite = np.asarray([[-2.159656e+06],[-2.111602e+07],[1.559791e+07]])
diferencia = satellite-receptor

In [37]:
diferencia

array([[  -654311.82588334],
       [-17718939.0957993 ],
       [ 10429852.1633697 ]])

In [30]:
p2 = np.asarray([[1,0,0],[0,-1,0],[0,0,1]]) 
def R2(angle):
    angle = angle*math.pi/180
    return np.asarray([[math.cos(angle),0,-math.sin(angle)],[0,1,0],[math.sin(angle),0,math.cos(angle)]])
def R3(angle):
    angle = angle*math.pi/180
    return np.asarray([[math.cos(angle),math.sin(angle),0],[-math.sin(angle),math.cos(angle),0],[0,0,1]])

In [31]:
Resultado = np.dot(p2,(np.dot(R2(latitud_r-90),np.dot(R3(longitud_r-180),diferencia))))

In [32]:
magnitude = (Resultado[0,0]**2+Resultado[1,0]**2+Resultado[2,0]**2)**.5
elevacion=math.asin(Resultado[2,0]/magnitude)*180/math.pi

In [33]:
elevacion

61.372950176685436